# <u>GERADOR MORNING CALLS</u> 

O programa é uma exploração inicial da API da OpenAI. A ideia é gerar um morning call do mercado através de demais fontes

#### O que o programa faz:
1. Acessa o conteúdo das morning calls da Órama e XP;
2. Resume separadamente cada morning call;
3. Usa o ChatGPT para criar um texto no formato de morning call baseado nos tópicos;
4. Separa o texto no contexto do Brasil e exterior e exclui parágrafos não relevantes.

Pontos de atenção:

* O ChatGPT limita cada completion por um número de tokens (acho que 4000, somando perguntas e respostas). Até então este valor nunca foi ultrapassado, provavelmente algo maior que isso é até muita coisa para ler de qualquer forma;
* O ChatGPT limita o número de completions em 3 por minuto, então caso rode mais que isso ele da um erro, é só esperar um tempo e rodar novamente.
* Os resultados ainda estão longe de automatizados, são necessários muitos ajustes manuais de classificação de Brasil/externo e coerência.
* Alguns dados são divulgados depois que as fontes divulgam o morning call. Nestes casos é necessário pesquisar manualmente sobre a divulgação e pode ser inputados na função "one_paragraph_summary" que resume em um parágrafo.

# 0. Import Libraries and Preparation

In [1]:
import re
import time 
import locale
import keyboard
import pyautogui
import feedparser
import clipboard
import pandas as pd
import numpy as np
import yfinance as yf
import requests

from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from newspaper import Article

## OpenAI Data
import openai

openai.api_key = 'YOUR OPENAI KEY'

## Date
today = datetime.today().strftime('%m/%d/%Y')
print(today)

ModuleNotFoundError: No module named 'keyboard'

In [ ]:
def get_text_piece(text, starts, ends):
    """
    Get paragraphs from the text that are
    between the string "starts" and "ends"
    (not included)
    
    """
    return text.split(starts)[1].split(ends)[0]

In [ ]:
def newspaper_text_extraction(file_url):
    """
    Extract text from URL
    
    """
    article = Article(file_url)
    article.download()
    article.parse()
    
    text = article.text
    
    ## Correction for XP case
    if article.text.startswith('SOBRE O EVENTO') or article.text.startswith('Disclaimer'):
        text = get_text_piece(article.html,
                              '<h2 class="wp-block-heading" id="destaque-do-dia-ou-intertitulo-1">',
                              'Empresas</span></h2>')
        
        text = re.sub(r'<.*?>', '', text)
        text = text.replace('\t', '')
        text = re.sub(r'\n{3,}', '\n\n', text)
    
    return text

## 1. Getting Morning Call Sources

In [ ]:
## Extract Órama Morning Call
link_orama = 'https://analise.orama.com.br/panorama-diario-3/'
MC_orama = newspaper_text_extraction(link_orama)


## Extract XP Morning Call
link_xp_base = 'https://conteudos.xpi.com.br/morning-call/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

request = requests.get(link_xp_base, headers=headers)
soup = BeautifulSoup(request.content, 'html.parser')

link_xp = soup.find('a', class_='bloco-materia box-rounded')['href']

MC_xp = newspaper_text_extraction(link_xp)
MC_xp = MC_xp.split('\n\nEmpresas')[0]

# 2. Generate Topics

In [ ]:
def extract_interest_topics(text, words_out, range_min, range_max):
    paragraphs = text.split('\n\n')[range_min:range_max]
    final_paragraphs = []
    
    for paragraph in paragraphs:
        keep_paragraph = True
        
        for word in words_out:
            if word in paragraph.lower():
                keep_paragraph = False
        
        if keep_paragraph:
            final_paragraphs.append(paragraph)

    return '\n\n'.join(final_paragraphs)

## 2.1. This Morning

In [ ]:
completion = openai.ChatCompletion.create(model='gpt-3.5-turbo',
                                          messages=[{'role': 'system',
                                                     'content': 'You are an Asset Manager and Investment Advisor'},
                                                    {'role': 'user',
                                                     'content': 'Baseado no texto:\n\n' + MC_orama + '\n\nEscreva um \
                                                                 resumo comentando sobre o cenário de forma completa'}])

summary_orama = completion['choices'][0]['message']['content']
time.sleep(30)

In [ ]:
print(summary_orama)

O cenário econômico e político observado nesta manhã está fortemente influenciado por eventos como as decisões de política monetária nos Estados Unidos e no Brasil. Enquanto se aguarda um possível recuo de 50 bps na taxa básica de juros pelo Banco Central do Brasil, as expectativas para o Fed dos EUA são de manutenção. Nas bolsas, a Ásia fechou majoritariamente em alta, impulsionada pela manutenção das taxas de juros na China e à espera do anúncio do Fed. Já na Europa, as bolsas operam sem direção única, influenciadas por quedas no setor de luxo e aguardando a decisão do Fed. No mercado de commodities, o rendimento do T-Note de 10 anos está em 4,28%, os contratos futuros do Brent recuam e o ouro também apresenta queda. O Bitcoin é negociado em alta. Nos EUA, eventos importantes incluem os estoques de Petróleo Bruto, a decisão de política monetária do FOMC e a coletiva de imprensa do FOMC, enquanto no Brasil destaca-se a decisão do COPOM. No mercado brasileiro, o Ibovespa avançou impuls

In [ ]:
completion = openai.ChatCompletion.create(model='gpt-3.5-turbo',
                                          messages=[{'role': 'system',
                                                     'content': 'You are an Asset Manager and Investment Advisor'},
                                                    {'role': 'user',
                                                     'content': 'Baseado no texto:\n\n' + MC_xp + '\n\nEscreva um \
                                                                 resumo comentando sobre o cenário de forma completa'}])

summary_xp = completion['choices'][0]['message']['content']
time.sleep(30)

In [ ]:
print(summary_xp)

Resumo:

No cenário financeiro recente, o Ibovespa fechou com leve alta impulsionado por commodities, enquanto a Embraer teve destaque positivo e a Magalu registrou desempenho negativo. As taxas futuras de juros tiveram queda, alinhadas com os Treasuries dos EUA, enquanto os mercados globais apresentaram variações mistas. Na economia, o PBoC manteve taxas de juros na China, os EUA viram uma forte recuperação na construção de casas e no Brasil, as expectativas de inflação para 2024 e 2025 estão acima da meta. Na Super Quarta-feira, o FOMC deve manter as taxas de juros nos EUA, com possibilidade de cortes a partir de julho. No Brasil, espera-se que o Copom reduza a taxa Selic em 50 pontos-base, refletindo a dinâmica da inflação e a atividade econômica mais forte. Os riscos de curto prazo estão inclinados para cima, com projeções de cortes adicionais nas taxas de juros até atingir 9,0%.


In [ ]:
completion = openai.ChatCompletion.create(model='gpt-3.5-turbo',
                                          messages=[{'role': 'system',
                                                     'content': 'You are an Asset Manager and Investment Advisor'},
                                                    {'role': 'user',
                                                     'content': 'Baseado nos resumos:\n\n' + summary_orama + '\n\n' +
                                                                 summary_xp + '\n\nEscreva uma carta comentando sobre \
                                                                 o cenário de forma completa'}])

todays_text = completion['choices'][0]['message']['content']
time.sleep(15)

In [ ]:
print(todays_text)

Prezado cliente,

Espero que esteja bem. Gostaria de abordar o atual cenário econômico e político que tem impactado os mercados financeiros recentemente. 

Observamos que as decisões de política monetária tanto nos Estados Unidos quanto no Brasil têm sido um ponto central de atenção para investidores e analistas. Enquanto aguardamos possíveis movimentos de redução na taxa básica de juros pelo Banco Central do Brasil, as expectativas em relação ao Fed dos EUA apontam para a manutenção das taxas. Esses eventos têm gerado impactos nas bolsas de valores ao redor do mundo, com a Ásia fechando majoritariamente em alta, impulsionada pela estabilidade das taxas de juros na China e pela expectativa em relação ao anúncio do Fed.

Na Europa, as bolsas operam sem uma direção clara, influenciadas por quedas no setor de luxo e também aguardando a decisão do Fed. No mercado de commodities, observamos movimentações interessantes, como o rendimento do T-Note de 10 anos em 4,28%, a queda nos contratos f

In [ ]:
todays = extract_interest_topics(todays_text, [], 2, -3)
print(todays)

Observamos que as decisões de política monetária tanto nos Estados Unidos quanto no Brasil têm sido um ponto central de atenção para investidores e analistas. Enquanto aguardamos possíveis movimentos de redução na taxa básica de juros pelo Banco Central do Brasil, as expectativas em relação ao Fed dos EUA apontam para a manutenção das taxas. Esses eventos têm gerado impactos nas bolsas de valores ao redor do mundo, com a Ásia fechando majoritariamente em alta, impulsionada pela estabilidade das taxas de juros na China e pela expectativa em relação ao anúncio do Fed.

Na Europa, as bolsas operam sem uma direção clara, influenciadas por quedas no setor de luxo e também aguardando a decisão do Fed. No mercado de commodities, observamos movimentações interessantes, como o rendimento do T-Note de 10 anos em 4,28%, a queda nos contratos futuros do Brent e no preço do ouro. O Bitcoin, por sua vez, tem apresentado um desempenho positivo.

Eventos importantes nos EUA incluem os estoques de Petr

# 3. Generate Morning Call

In [ ]:
def separate_topic_paragraphs(text, BR_keywords):
    paragraphs = text.split('\n\n')
    BR_topics = []
    EX_topics = []
    
    for paragraph in paragraphs:
        keep_paragraph = False
        
        for keyword in BR_keywords:
            if keyword in paragraph.lower():
                keep_paragraph = True
                continue
                
        if keep_paragraph:
            BR_topics.append(paragraph)
        else:
            EX_topics.append(paragraph)
    
    return BR_topics, EX_topics

In [ ]:
BR_topics, EX_topics = separate_topic_paragraphs(todays, ['brasil', 'brasília', 'lula', 'bolsonaro', 'pacheco', 'haddad'])
BR_topics = [paragraph.split('No Brasil, ')[1][0].upper() + 
             paragraph.split('No Brasil, ')[1][1:] if paragraph.startswith('No Brasil, ') \
                                                   else paragraph for paragraph in BR_topics]

BR_flag = '\U0001F1E7\U0001F1F7'
globe = '\U0001F310'

BR_text = '\n\n'.join([BR_flag + ' ' + topic for topic in BR_topics])
EX_text = '\n\n'.join([globe  + ' ' + topic for topic in EX_topics])

In [ ]:
MC = '*Destaques do dia*\n\n' + BR_text + '\n\n' + EX_text

print(MC)

*Destaques do dia*

🇧🇷 Observamos que as decisões de política monetária tanto nos Estados Unidos quanto no Brasil têm sido um ponto central de atenção para investidores e analistas. Enquanto aguardamos possíveis movimentos de redução na taxa básica de juros pelo Banco Central do Brasil, as expectativas em relação ao Fed dos EUA apontam para a manutenção das taxas. Esses eventos têm gerado impactos nas bolsas de valores ao redor do mundo, com a Ásia fechando majoritariamente em alta, impulsionada pela estabilidade das taxas de juros na China e pela expectativa em relação ao anúncio do Fed.

🇧🇷 Eventos importantes nos EUA incluem os estoques de Petróleo Bruto, a decisão de política monetária do FOMC e a coletiva de imprensa do FOMC, enquanto no Brasil destaca-se a decisão do COPOM. Internamente, o Relatório Focus do Banco Central indicou ajustes nas projeções para inflação, PIB e câmbio, refletindo a dinâmica da economia brasileira. Além disso, notícias políticas, como a regulamentação d